# GoPhish feature extraction

### Importing libraries

In [3]:
import re #regex
import string
import logging

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

### Importing csv file

Edited csv file to only show a value for 1 if the url is malicious, defaced or has malware. The URL will be marked as 0 if it is benign

In [4]:
url_data = pd.read_csv('goPhish_url_data.csv')
url_data.head()

,url,type
0,br-icloud.com.br,1
1,mp3raid.com/music/krizz_kaliko.html,0
2,bopsecrets.org/rexroth/cr/1.htm,0
3,http://www.garage-pirenne.be/index.php?option=...,1
4,http://adventure-nicaragua.net/index.php?optio...,1


In [12]:
url_data.info() #Displaying some information about the csv

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 651191 entries, 0 to 651190
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   url     651191 non-null  object
 1   type    651191 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 9.9+ MB


### Checking for null values in dataset

In [17]:
url_data.isnull().sum() #isnull() returns boolean value, .sum() returns the sum of the values

url     0
type    0
dtype: int64

### Extracting count of each value

In [15]:
count = url_data['type'].value_counts()
print(count)

type
0    428103
1    223088
Name: count, dtype: int64


### Normalising the data

Removing 'www.' from the url to normalise the data and extract the featues necessary to detect. 

- 'regex=True' uses the regular expression pattern to find the 'www.' in the whole url string, otherwise it would only find the first instance in the string

In [6]:
url_data['url'] = url_data['url'].replace('www.','', regex=True)

url_data.head()

,url,type
0,br-icloud.com.br,1
1,mp3raid.com/music/krizz_kaliko.html,0
2,bopsecrets.org/rexroth/cr/1.htm,0
3,http://garage-pirenne.be/index.php?option=com_...,1
4,http://adventure-nicaragua.net/index.php?optio...,1


In [5]:
url_data["type"].value_counts()

type
0    428103
1    223088
Name: count, dtype: int64

### Extracting url length

In [7]:
def get_url_length(url):
    prefixes = ['http://','https://'] #Defining common prefixes for URLs
    
    for prefix in prefixes: #Loop to exclude prefix in length of URL
        if url.startswith(prefix):
            url = url[len(prefix):]
    
    url = url.replace('www.','')
    
    return len(url)

In [8]:
url_data['url_len'] = url_data['url'].apply(lambda x: get_url_length(str(x))) #Adding column to show URL length

In [9]:
url_data.head()

,url,type,url_len
0,br-icloud.com.br,1,16
1,mp3raid.com/music/krizz_kaliko.html,0,35
2,bopsecrets.org/rexroth/cr/1.htm,0,31
3,http://garage-pirenne.be/index.php?option=com_...,1,77
4,http://adventure-nicaragua.net/index.php?optio...,1,228
